# <div align="center" style="color: #ff5733;">New Pos Addition and Mapping</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\pos_add_20240827_file2.csv")
df.shape

(1, 9)

In [3]:
# df = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\POS_mapping_data May042024.xlsx", sheet_name = 'Sheet1')
# df.head(10)

In [4]:
df.head()

,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,TERRITORY,TSM,REGION,Brand
0,FC Home Center Las Pinas Event POS,Standalone,POS Operational,"August 26, 2024",MFXKULW,Alabang,Grace Belarmino,South Luzon,FC Home


In [5]:
df.rename(columns = {'Brand': 'BrandStores'}, inplace = True)
df.columns

Index(['POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY',
       'TERRITORY', 'TSM', 'REGION', 'BrandStores'],
      dtype='object')

In [6]:
df['BrandStores'].value_counts()

BrandStores
FC Home    1
Name: count, dtype: int64

Check whether we have duplicate purple key in the data provided by the SIL team

In [7]:
df['PURPLE KEY'].value_counts()

PURPLE KEY
MFXKULW    1
Name: count, dtype: int64

In [8]:
df = df[['BrandStores', 'POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY', 'REGION', 'TSM']]
df

,BrandStores,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM
0,FC Home,FC Home Center Las Pinas Event POS,Standalone,POS Operational,"August 26, 2024",MFXKULW,South Luzon,Grace Belarmino


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   BrandStores      1 non-null      object
 1   POS NAME         1 non-null      object
 2   STORE LOCATION   1 non-null      object
 3   STATUS           1 non-null      object
 4   DATE ACTIVATION  1 non-null      object
 5   PURPLE KEY       1 non-null      object
 6   REGION           1 non-null      object
 7   TSM              1 non-null      object
dtypes: object(8)
memory usage: 192.0+ bytes


In [10]:
df.rename(columns={'POS NAME':'POS', 'DATE ACTIVATION':'DateOpened', 'STORE LOCATION': 'StoreLocation', 'STATUS':'Status', 'PURPLE KEY':'PurpleKey', 'REGION':'Region'}, inplace = True)

In [11]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,FC Home,FC Home Center Las Pinas Event POS,Standalone,POS Operational,"August 26, 2024",MFXKULW,South Luzon,Grace Belarmino


In [12]:
from datetime import datetime
df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%d-%m-%Y')
# Format the datetime object to YYYY-MM-DD
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')

ValueError: time data "August 26, 2024" doesn't match format "%d-%m-%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [13]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%B %d, %Y')


In [14]:
print(df['DateOpened'].dtype)
# datetime64[ns]
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

datetime64[ns]
2024-08-26 00:00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BrandStores    1 non-null      object
 1   POS            1 non-null      object
 2   StoreLocation  1 non-null      object
 3   Status         1 non-null      object
 4   DateOpened     1 non-null      object
 5   PurpleKey      1 non-null      object
 6   Region         1 non-null      object
 7   TSM            1 non-null      object
dtypes: object(8)
memory usage: 192.0+ bytes


In [15]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%d-%b-%y')

ValueError: time data "2024-08-26" doesn't match format "%d-%b-%y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [16]:
print(df['DateOpened'].dtype)
# datetime64[ns]

object


In [17]:
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00

2024-08-26


In [18]:
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

AttributeError: Can only use .dt accessor with datetimelike values

In [19]:
df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%Y-%m-%d')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   BrandStores    1 non-null      object        
 1   POS            1 non-null      object        
 2   StoreLocation  1 non-null      object        
 3   Status         1 non-null      object        
 4   DateOpened     1 non-null      datetime64[ns]
 5   PurpleKey      1 non-null      object        
 6   Region         1 non-null      object        
 7   TSM            1 non-null      object        
dtypes: datetime64[ns](1), object(7)
memory usage: 192.0+ bytes


In [21]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,FC Home,FC Home Center Las Pinas Event POS,Standalone,POS Operational,2024-08-26,MFXKULW,South Luzon,Grace Belarmino


In [22]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
try:
    d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
    print("Table still exists")
except:
    print("Table dropped")

Query is running:   0%|          |Table dropped


In [23]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1
as 
select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;
"""
client.query(sq)



QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=18ac52f7-6f9a-42ce-89b4-bd36521be853>

In [24]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of d is:\t{d.shape}")

Job ID 748f95c1-c1f7-4bd3-a61d-5bf98baf8091 successfully executed: 100%|██████████|
Query is running:   0%|          |
Downloading: 100%|██████████|
The shape of d is:	(922, 8)


In [25]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""

dfsil = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID ea36f34a-c503-448b-94ef-4340520d75e4 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [26]:
dfsil.BrandStores.value_counts().sort_index()

BrandStores
Affordables                     2
Ansons                         15
ArkHome                         4
Automatic Centre               11
Chaiver Telecom                 9
Chrisrence Home Appliances      2
Dizon Furniture                 1
ESLA                           51
FC Home                       110
Faith Glorious                  9
Faith Glorious Fit              8
Gadgetmate                     10
Hardware Sugar                  2
Hazrat                          6
Home Along                     96
Home Factory                    2
Homeworks                      15
JCE Fone                       15
JSL                             7
JTG                             1
Jhunpyo                        12
Junliz                          8
KUM Home Center                 4
Kservico                       10
MemoXpress                     58
PHPS                            2
Power Mac                       5
Power Mac Center                5
Presnet                        21
Pr

In [27]:
dfsil.shape

(922, 8)

In [28]:
dfsil.columns

Index(['BrandStores', 'POS', 'StoreLocation', 'Status', 'DateOpened',
       'PurpleKey', 'Region', 'TSM'],
      dtype='object')

In [29]:
dfsil[dfsil['PurpleKey'] =='MFXKULW']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM


In [30]:
# Set table_id to the ID of the table to append to.
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND"
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  

job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 923 rows and 8 columns to prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1


In [31]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f"The shape of the dataframe is: {df.shape}")

Job ID 2a76f49b-95ec-4ad5-a953-700f451aec53 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is: (923, 8)


In [32]:
df.tail(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
913,FC Home,FC Home Center Mamatid,Standalone,POS Operational,2023-09-14,MC31AV7,South Luzon 2,Christian Klide Alberto
914,FC Home,FC Home Center Sta Cruz,Standalone,POS Operational,2023-09-14,M1ZYHKU,South Luzon 2,Christian Klide Alberto
915,FC Home,FC Home Center Binan,Central Mall Laguna,POS Operational,2023-09-14,MYRK3EJ,South Luzon 2,Christian Klide Alberto
916,FC Home,FC Home Center San Pablo,Standalone,POS Operational,2023-09-14,MADP0JZ,South Luzon 2,Christian Klide Alberto
917,FC Home,FC Home Center Siniloan,Standalone,POS Operational,2023-09-15,MANB6XQ,South Luzon 2,Christian Klide Alberto
918,Workstation,Workstation Home Appliances Target Mall,Target Mall,POS Operational,2023-10-13,M8I4QGT,South Luzon 2,Christian Klide Alberto
919,Workstation,Workstation Home Appliances Central Mall,Central Mall,POS Operational,2023-10-13,MTNCOR5,South Luzon 2,Christian Klide Alberto
920,Workstation,Workstation Home Appliances Pavilion Mall 1,Pavillon Mall,POS Operational,2023-10-13,MN8MWPJ,South Luzon 2,Christian Klide Alberto
921,Xtreme,Xtreme Appliances Calamba,Carmel Mall,POS Operational,2023-10-15,MUBPW04,South Luzon 2,Christian Klide Alberto
922,FC Home,FC Home Center Las Pinas Event POS,Standalone,POS Operational,2024-08-26,MFXKULW,South Luzon,Grace Belarmino


In [33]:
df['BrandStores'].value_counts().sort_index()

BrandStores
Affordables                     2
Ansons                         15
ArkHome                         4
Automatic Centre               11
Chaiver Telecom                 9
Chrisrence Home Appliances      2
Dizon Furniture                 1
ESLA                           51
FC Home                       111
Faith Glorious                  9
Faith Glorious Fit              8
Gadgetmate                     10
Hardware Sugar                  2
Hazrat                          6
Home Along                     96
Home Factory                    2
Homeworks                      15
JCE Fone                       15
JSL                             7
JTG                             1
Jhunpyo                        12
Junliz                          8
KUM Home Center                 4
Kservico                       10
MemoXpress                     58
PHPS                            2
Power Mac                       5
Power Mac Center                5
Presnet                        21
Pr

In [34]:
# Find duplicate values in 'POS' column
duplicates = df[df.duplicated('POS', keep=False)]

# Print the entire rows for duplicate entries in 'POS'
if not duplicates.empty:
    print("Duplicate values in 'POS' column:")
    duplicates.to_csv("Temp.csv")
else:
    print("No duplicate values found in 'POS' column.")
    
dd = pd.read_csv("Temp.csv")
dd.sort_values(by='POS')

Duplicate values in 'POS' column:


,Unnamed: 0,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,7,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,MV4UO0X,NaN,NaN
1,8,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,M9ZK6IU,NaN,NaN
31,886,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MGP2AQI,NCR North,Ben Antonio Lacsamana
30,885,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MN3GL66,NCR North 2,Ben Antonio Lacsamana
32,891,Home Along,Home Along Malanday,Standalone,POS Operational,2022-07-11,MNBW4RZ,NCR North,Ben Antonio Lacsamana
33,892,Home Along,Home Along Malanday,NaN,NaN,2022-07-11,MURFAIV,NCR North,Ben Antonio Lacsamana
21,803,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MU82X9Z,NCR South,Klein Justine Tamayo
22,805,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MDGYHQJ,NCR South,Klein Justine Tamayo
25,819,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MDO3KI4,NCR South,Klein Justine Tamayo
26,820,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MRGP0WQ,NCR South,Klein Justine Tamayo


In [35]:
pd.set_option('display.max_rows', None)
# Count the occurrences of each value in the 'PurpleKey' column
purple_key_counts = df['PurpleKey'].value_counts()

# Filter to include only values with count >= 2
filtered_counts = purple_key_counts[purple_key_counts >= 2]

# Display the filtered counts
print(filtered_counts)


Series([], Name: count, dtype: int64)


In [36]:
df[df['PurpleKey']== 'MFXKULW']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
922,FC Home,FC Home Center Las Pinas Event POS,Standalone,POS Operational,2024-08-26,MFXKULW,South Luzon,Grace Belarmino


In [37]:
sq = """drop table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2648d038-9e70-42e2-b5b2-e9d088b17755>

In [38]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING as select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=d1f4ec8f-7ac8-41cd-8e06-f04968386ff0>

In [39]:
client.query("commit")

QueryJob<project=prj-prod-dataplatform, location=US, id=96a1743b-3c60-47f1-9ae2-e379b76c09e9>

In [40]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df.shape

Job ID 4b918337-4a90-470c-a40c-9648b38d578f successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(923, 8)

In [41]:
df[df['BrandStores']=='VMax Mobile'].sort_values(by='DateOpened', ascending = False)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
544,VMax Mobile,Delle General Merch Target Mall,Target Mall Balibago,For Opening,2024-08-02,MLX9FJP,South Luzon,Aubrey Lalong Isip
545,VMax Mobile,Realme Target Mall Balibago,Target Mall Balibago,For Opening,2024-08-02,MXACV0S,South Luzon,Aubrey Lalong Isip
546,VMax Mobile,Vmax Waltermart Cabuyao,Waltermart Cabuyao,For Opening,2024-08-02,MKPNC4B,South Luzon,Aubrey Lalong Isip
547,VMax Mobile,St Vincent Telecom Target Mall,Target Mall Starosa,For Opening,2024-08-02,MVTZGDH,South Luzon,Aubrey Lalong Isip
538,VMax Mobile,Vmax Central Binan,Central Mall Binan,POS Operational,2024-07-27,MZEO85E,South Luzon,Aubrey Lalong Isip
539,VMax Mobile,Vmax Waltermart Sta Rosa,Waltermart Starosa,POS Operational,2024-07-27,MJ0WZAK,South Luzon,Aubrey Lalong Isip
540,VMax Mobile,Vmax Central Binan 2,Central Mall Binan,POS Operational,2024-07-27,ME4NBT7,South Luzon,Aubrey Lalong Isip
541,VMax Mobile,Lets Mobile Phone and accessories Store,Divimall Cabuyao,POS Operational,2024-07-27,MSNZ3XL,South Luzon,Aubrey Lalong Isip
542,VMax Mobile,Delle General Merch Harmony Mall,Harmony Village Mall,POS Operational,2024-07-27,M3YOW2R,South Luzon,Aubrey Lalong Isip
543,VMax Mobile,Vmax Central Balibago,Victory Central Mall Balibago,POS Operational,2024-07-27,MWN8FVI,South Luzon,Aubrey Lalong Isip


In [42]:
df.head(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,FC Home,FC Home Center Balibago,LOT 5 BLK 9 GARNET ST. RHINELAND COMMERCIAL CO...,Operational,2023-08-01,MOG243Y,South Luzon 2,Klide Alberto
1,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center San ...,Standalone,POS Operational,2024-08-02,MB3YSES,CAMANAVA,Paul Palonpon
2,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Pasig,Standalone,POS Operational,2024-08-02,MOJ34ME,ENCR,Martin Lin
3,Home Along,Home Along Talavera,Standalone,POS Operational,2024-08-15,MZKJOGD,North Luzon,Rhea Geronimo
4,JCE Fone,Infinix Concept Store Waltermart Tanauan,Waltermart Tanauan,For Opening,2024-08-24,MZ3HN9Y,South Luzon,Jan Jan Dela Cruz
5,JCE Fone,JCE Fonehub Waltermart Tanauan,Waltermart Tanauan,For Opening,2024-08-24,MK4WN52,South Luzon,Jan Jan Dela Cruz
6,Xtreme,Xtreme Appliances Anabu,None,None,2022-08-27,MR2YDE8,None,None
7,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,MV4UO0X,None,None
8,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,M9ZK6IU,None,None
9,Home Along,Home Along Sumulong,None,None,2022-07-12,M2G78K4,None,None


In [43]:
df.sort_values('DateOpened', ascending=False).head(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
556,Chaiver Telecom,Chaiver Telecom Alaminos,Standalone,For Opening,2024-08-29,MC8F23C,South Luzon,Aubrey Lalong Isip
145,Chaiver Telecom,Raisen Telecom San Antonio Quezon,Standalone,For Opening,2024-08-29,MIEY03Q,South Luzon,Penfrancia Ayo
555,Chaiver Telecom,Raisen Telecom Stall 33A Shopping Mall San Pablo,Shopping Mall San Pablo,For Opening,2024-08-29,MZJ7RCQ,South Luzon,Aubrey Lalong Isip
554,Chaiver Telecom,Raisen Telecom Main San Pablo,Shopping Mall San Pablo,For Opening,2024-08-29,MHQET09,South Luzon,Aubrey Lalong Isip
144,Chaiver Telecom,Raisen Telecom Tiaong,Standalone,For Opening,2024-08-29,M8Q6CW5,South Luzon,Penfrancia Ayo
192,FC Home,FC Home Center Las Pinas Event POS,Standalone,POS Operational,2024-08-26,MFXKULW,South Luzon,Grace Belarmino
142,JCE Fone,Infinix LCC Goa,LCC Mall Goa,For Opening,2024-08-24,M23CNPV,South Luzon,Penfrancia Ayo
553,Chaiver Telecom,Chaiver Telecom Stall 58 Shopping Mall San Pablo,Shopping Mall San Pablo,POS Operational,2024-08-24,M0YTZ5U,South Luzon,Aubrey Lalong Isip
552,Chaiver Telecom,ICRS Cellphone and Accessories Ultimart San Pablo,Ultimart San Pablo,POS Operational,2024-08-24,MNTYONE,South Luzon,Aubrey Lalong Isip
551,JCE Fone,Infinix Central Mall Sta Cruz,Central Mall Sta Cruz,For Opening,2024-08-24,MLQ6VA0,South Luzon,Aubrey Lalong Isip


In [44]:
df[df['PurpleKey'] =='MFXKULW']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
192,FC Home,FC Home Center Las Pinas Event POS,Standalone,POS Operational,2024-08-26,MFXKULW,South Luzon,Grace Belarmino


In [45]:
df.groupby(['BrandStores', 'DateOpened'])['PurpleKey'].nunique()

BrandStores                 DateOpened
Affordables                 2024-01-22      1
                            2024-04-12      1
Ansons                      2024-04-15     15
ArkHome                     2024-06-07      4
Automatic Centre            2023-07-15      6
                            2023-07-16      1
                            2023-07-22      4
Chaiver Telecom             2024-08-24      4
                            2024-08-29      5
Chrisrence Home Appliances  2024-07-13      2
Dizon Furniture             2022-07-23      1
ESLA                        2024-02-15     30
                            2024-04-22     21
FC Home                     2023-08-01     48
                            2023-09-01     20
                            2023-09-05      1
                            2023-09-14     25
                            2023-09-15     11
                            2023-11-10      1
                            2024-01-15      1
                            2024-03-26   

In [46]:
df[df['BrandStores']=='Rulls']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
13,Rulls,MBC Gaisano Inline Binangonan,Gaisano Inline Binangonan,For Opening,2024-04-15,M38S4QA,NCR East,Albee David
14,Rulls,Mobile Craze Town Center Mall Tanay,Craze Town Center Mall Tanay,For Opening,2024-04-15,M3ED6SF,NCR East,Albee David
15,Rulls,MBC Montalban Town Center,Montalban Town Center,For Opening,2024-04-15,MI9KLNA,NCR East,Albee David
16,Rulls,Mobilecraze K2 PG Marikina,Puregold Marikina,For Opening,2024-04-15,MUFDX4A,NCR East,Albee David
18,Rulls,Rulls Unimart Capitol,Unimart Capitol Common,For Opening,2024-04-15,MNGI12Q,NCR East,Albee David
19,Rulls,Fun Mobile Ayala Marikina,Ayala Malls Marikina,For Opening,2024-04-15,MK4CZP2,NCR East,Albee David
20,Rulls,Mobilecraze K3 PG Marikina,Puregold Marikina,For Opening,2024-04-15,MW6HOE7,NCR East,Albee David
23,Rulls,Techbox Antipolo,Robinsons Antipolo,For Opening,2024-04-15,M8FWM1N,NCR East,Albee David
24,Rulls,Oppo Robinsons Metro East,Robinsons Metro East,For Opening,2024-04-15,MTEHX0R,NCR East,Albee David
25,Rulls,Mobile Craze PG Tanay Rizal,Puregold Tanay Rizal,For Opening,2024-04-15,MFCWRJQ,NCR East,Albee David


In [47]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\POS_mapping_data.xlsx", index = False)

In [48]:
df[df['PurpleKey']=='MFXKULW']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
192,FC Home,FC Home Center Las Pinas Event POS,Standalone,POS Operational,2024-08-26,MFXKULW,South Luzon,Grace Belarmino
